In [201]:
# Import Dependencies
import pandas as pd

In [202]:
measurements_path="Resources/hawaii_measurements.csv"
station_path="Resources/hawaii_stations.csv"

In [203]:
measurements_df = pd.read_csv(measurements_path)
station_df = pd.read_csv(station_path)
measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


In [204]:
measurements_df.count()

station    19550
date       19550
prcp       18103
tobs       19550
dtype: int64

In [205]:
measurement_clean_df = measurements_df.dropna(axis=0, how='any')
station_clean_df = station_df.dropna(axis=0, how='any')

In [206]:
measurement_clean_df.to_csv("./Resources/hawaii_measurements_clean.csv", index=False)
station_clean_df.to_csv("./Resources/hawaii_station_clean.csv", index=False)

In [207]:
measurement_clean_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
5,USC00519397,2010-01-07,0.06,70


In [208]:
station_clean_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [209]:

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Date


In [210]:
# Create the Measurement class
### BEGIN SOLUTION
class Measurements(Base):
    __tablename__ = 'Measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Integer)
### END SOLUTION

# Create the Station class
### BEGIN SOLUTION
class Stations(Base):
    __tablename__ = 'Station'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
### END SOLUTION


# Create a connection to a SQLite database
### BEGIN SOLUTION
engine = create_engine('sqlite:///hawaii.sqlite')
### END SOLUTION


# Create the garbage_collection table within the database
Base.metadata.create_all(engine)
# To push the objects made and query the server we use a Session object
session = Session(bind=engine)
# Commit the objects to the database
session.commit()



In [211]:
# Create the inspector and connect it to the engine
from sqlalchemy import inspect
inspector = inspect(engine)


In [212]:
# Collect the names of tables within the database
inspector.get_table_names()


['Measurement', 'Station']

In [213]:
# Using the inspector to print the column names within the 'dow' table and its types
columns = inspector.get_columns('Measurement')
print(columns)

[{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'station', 'type': VARCHAR(length=255), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'date', 'type': DATE(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'prcp', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'tobs', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}]


In [214]:
engine.execute('SELECT * FROM Station LIMIT 10').fetchall()


[]

In [215]:
from datetime import datetime

#station	date	prcp	tobs
for index, rows in measurement_clean_df.iterrows():
    #print(datetime.strptime(rows['date'], "%Y-%m-%d"))
    measurement = Measurements(station=rows['station'], date=datetime.strptime(rows['date'],"%Y-%m-%d"), prcp=rows['prcp'], tobs=rows['tobs'])
    session.add(measurement)
    session.commit()


In [216]:
engine.execute('SELECT * FROM Measurement LIMIT 10').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70),
 (6, 'USC00519397', '2010-01-08', 0.0, 64),
 (7, 'USC00519397', '2010-01-09', 0.0, 68),
 (8, 'USC00519397', '2010-01-10', 0.0, 73),
 (9, 'USC00519397', '2010-01-11', 0.01, 64),
 (10, 'USC00519397', '2010-01-12', 0.0, 61)]

In [217]:
#station	name	latitude	longitude	elevation
for index, rows in station_clean_df.iterrows():
    #print(datetime.strptime(rows['date'], "%Y-%m-%d"))
    station = Stations(station=rows['station'], name=rows['name'], latitude=rows['latitude'], longitude=rows['longitude'],
                              elevation=rows['elevation'])
    session.add(station)
    session.commit()

In [219]:
engine.execute('SELECT count(*) FROM Measurement').fetchall()

[(18103,)]